<a href="https://colab.research.google.com/github/kangmg/compchem_with_colab/blob/main/ase_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install programs

In [1]:
# clear cell output
from IPython.display import clear_output

# 의존성 프로그램 설치
!apt install swig # version : 4.0.2

# openbabel 소스코드 설치
!wget -q https://github.com/openbabel/openbabel/archive/refs/tags/openbabel-3-1-1.tar.gz -O openbable.tar.gz
!tar vxf ./openbable.tar.gz
!rm ./openbable.tar.gz

# 소스코드 빌드 & 파이썬 바인딩
!mkdir ./build
%cd build
!cmake ../openbabel-openbabel-3-1-1 -DPYTHON_BINDINGS=ON -DRUN_SWIG=ON
clear_output()
!echo "%%%%%%%%     Compiling . . .  %%%%%%%%"
!make -j2
clear_output()
!echo "%%%%%%%%     Building . . .  %%%%%%%%"
!make install
%cd ..
clear_output()
!echo "openbabel successfully installed"

openbabel successfully installed


In [2]:
!pip -q install git+https://github.com/cclib/cclib.git    # compchem package
!pip -q install git+https://gitlab.com/ase/ase.git        # ase
!pip -q install git+https://github.com/kangmg/xtb_ase.git # ase_xtb

clear_output()
!echo "dependent packages are successfully installed!"

dependent packages are successfully installed!


In [3]:
# xtb installation
!wget -q https://github.com/grimme-lab/xtb/releases/download/v6.6.1/xtb-6.6.1-linux-x86_64.tar.xz -O ./xtb.tar.xz
!tar vxf ./xtb.tar.xz
!rm ./xtb.tar.xz
!mkdir xtb_tmp
clear_output()
!echo "xtb successfully installed!"

# xtb enviromental setting
import os
os.environ['PATH'] += ':/content/xtb-6.6.1/bin'

xtb successfully installed!


In [4]:
# torchani 설치
!pip -q install torchani
clear_output()
!echo "torchani successfully installed"

torchani successfully installed


In [5]:
# install ase
#!pip -q install git+https://gitlab.com/ase/ase

# lastest version mopac installation
!wget -q http://openmopac.net/mopac-22.1.1-linux.tar.gz -O ./mopac.tar.gz
!tar vxzf ./mopac.tar.gz
!rm ./mopac.tar.gz
clear_output()
!echo "mopac successfully installed"

# append mopac run file path
import os
os.environ['PATH'] += ':/content/mopac-22.1.1-linux/bin'

mopac successfully installed


In [6]:
# install aimnet2
!pip install git+https://github.com/kangmg/aimnet2ase.git
clear_output()
!echo "aimnet2 successfully installed"

aimnet2 successfully installed


In [7]:
# install rdkit
!pip install rdkit
clear_output()
!echo "rdkit successfully installed"

rdkit successfully installed


In [8]:
import rdkit
import aimnet2ase
import torchani
import xtb_ase
import ase
from openbabel import pybel
import openbabel

print(f"""
rdkit  : {rdkit.__version__}
aimnet2ase  : {aimnet2ase.__version__}
torchani  : {torchani.__version__}
xtb_ase  : {xtb_ase.__version__}
ase  : {ase.__version__}
openbabel  : {openbabel.__version__}
""")

/usr/local/lib/python3.10/dist-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")



rdkit  : 2023.09.6
aimnet2ase  : 0.0.1
torchani  : 2.2.4
xtb_ase  : 0.0.1
ase  : 3.23.0b1
openbabel  : 3.1.0



In [9]:
!echo -e "\033[34mOpenbable version\033[0m"
!obabel -V
!echo -e "\n\033[34mxTB version\033[0m"
!xtb --version
!echo -e "\n\033[34mMOPAC version\033[0m"
!mopac --version

Openbable version
Open Babel 3.1.0 -- May 26 2024 -- 19:35:01

xTB version
      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.6.1 (8d0f1dd) compiled by 'stahn@M-Bot' on 2023-08-01

normal termination of xtb

MOPAC version
MOPAC version 22.1.1 commit 82a1da4b97f994dbff2423f7991c17716d3a39ea


## openbabel ( charge setting issue ) / from xyz

In [10]:
from openbabel import pybel

"""
    Charge를 세팅해도 인식되지 않는 오류가 있음
    --> 확인 필요
"""

def openbabel_optimize(xyz_string:str, charge:int, forcefield:str)->str:
  """
    Parameters
    ----------
      - xyz_string(str) : xyz format string
      - charge(int) : molecular total charge
      - forcefield(str) : supported force field : ['uff', 'mmff94', 'ghemical']

    Returns
    -------
      - optimized xyz string(str)
  """
  # read xyz
  molecule = pybel.readstring(format="xyz", string=xyz_string)

  # set charge
  """
  To be implemented

  자동 세팅이라면 charge 구한 값이랑 같은지 확인하는 절차를 넣어야 함
  """

  # optmize using FF
  molecule.localopt(forcefield=forcefield)
  optimized_xyz = molecule.write("xyz")
  return optimized_xyz

In [13]:
# test

xyz_input = """4

B     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(openbabel_optimize(xyz_input, 0, "uff"))

4

B          0.16846        1.84165        1.03794
H         -0.17055        0.90757        1.67639
H         -0.18352        2.92603        1.34657
H          0.91960        1.69936        0.13763



## rdkit / smiles to sdf

In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles2mol(smiles:str)->str:
  """
    get mol format block from smiles
  """
  # smiles to mol block
  mol = Chem.MolFromSmiles(smiles)
  mol = Chem.AddHs(mol)

  # sanitize
  Chem.SanitizeMol(mol)

  # generate 3D embedded conformers
  num_conformers = 50
  params = AllChem.ETKDGv3() # embedding algorithm
  params.randomSeed = 42

  conformer_ids = AllChem.EmbedMultipleConfs(mol, numConfs=num_conformers)

  energies = []
  for conf_id in conformer_ids:
    # MMFF94 힘장 설정
    ff = AllChem.UFFGetMoleculeForceField(mol, confId=conf_id)

    # force field optimize
    ff.Minimize()
    energy = ff.CalcEnergy()
    energies.append((conf_id, energy))

  # get lowest conformer
  lowest_energy_idx, _ = min(energies, key=lambda x: x[-1])

  # mol format block
  mol_block = Chem.MolToMolBlock(mol, confId=lowest_energy_idx)

  return mol_block


In [16]:
def rdkit_optimize(smiles:str, charge:int, forcefield:str, mmffVariant=None)->str:
  """
  Description
  -----------
  rdkit geometry optimize using force field

  Parameters
  ----------
    - smiles(str) : smiles string
    - charge(int) : molecular total charge
    - forcefield(str) : supported force field : ['MMFF', 'UFF']
    - mmffVariant(str) : MMFF forcefield variants : ['MMFF94', 'MMFF94s']

  Returns
  -------
    - optimized xyz string(str)
  """
  # forcefield parameter check
  if forcefield == "MMFF" and mmffVariant not in ["MMFF94", "MMFF94s"]: raise ValueError(f'mmffVariant expected "MMFF94" or "MMFF94s", but got {mmffVariant}')

  # read smiles
  mol_block = smiles2mol(smiles)
  mol = Chem.MolFromMolBlock(mol_block, removeHs=False)

  # charge check
  Chem.rdPartialCharges.ComputeGasteigerCharges(mol)
  partial_chargies = [atom.GetDoubleProp('_GasteigerCharge') for atom in mol.GetAtoms()]
  charge_from_molecule = round(sum(partial_chargies))
  charge_from_input = charge

  if charge_from_molecule != charge_from_input:
    print("Incosistency of charge")
    return None

  # optimize
  if forcefield == 'UFF':
      status = AllChem.UFFOptimizeMolecule(mol, maxIters=2000)
  elif forcefield == 'MMFF':
      status = AllChem.MMFFOptimizeMolecule(mol, mmffVariant, maxIters=2000)
  else:
      raise ValueError(f"{forcefield} is supported. Use 'UFF' or 'MMFF'.")

  # optimize status check
  if status == 1:
     print("Optimizer : Not converged in 2000 iteractions.")
     return None
  elif status == -1:
     print("Optimizer : Fail to optimize, status code : -1")
     return None

  optimized_xyz = Chem.MolToXYZBlock(mol)
  return optimized_xyz

In [17]:
# test

smiles = "CC1=CC(=O)/C(=C(/C)\[O-])/C(=O)O1"

print(rdkit_optimize(smiles, -1, "MMFF", "MMFF94s"))

19

C      3.160456    0.508230   -0.468472
C      1.801519   -0.023702   -0.152867
C      1.329498   -1.176282   -0.637112
C     -0.020043   -1.610504   -0.239202
O     -0.275473   -2.808919   -0.227811
C     -0.894151   -0.487054    0.199174
C     -2.153162   -0.345135   -0.297208
C     -2.986088    0.864427    0.114711
O     -2.793774   -1.046531   -1.129284
C     -0.247332    0.531404    1.065041
O     -0.796003    1.148896    1.972195
O      1.073165    0.769049    0.763422
H      3.079504    1.491096   -0.943462
H      3.711736   -0.150197   -1.147732
H      3.748229    0.615006    0.448800
H      1.885397   -1.841424   -1.282325
H     -2.401239    1.790367    0.064086
H     -3.857480    1.035352   -0.533031
H     -3.364862    0.735822    1.135777



## openbabel / from smiles

In [28]:
def openbabel_optimize(smiles:str, charge:int, forcefield:str)->str:
  """
  Description
  -----------
  openbabel geometry optimize using force field

  Parameters
  ----------
    - smiles(str) : smiles string
    - charge(int) : molecular total charge
    - forcefield(str) : supported force field : ['uff', 'mmff94', 'mmff94s', 'ghemical', 'gaff']

  Returns
  -------
    - optimized xyz string(str)
  """
  # read smiles
  mol_block = smiles2mol(smiles)
  molecule = pybel.readstring(format="mol", string=mol_block)

  # set charge
  charge_from_molecule = molecule.charge
  charge_from_input = charge

  if charge_from_molecule != charge_from_input:
    print("Incosistency of charge")
    return None

  # optmize using FF
  molecule.localopt(forcefield=forcefield)
  optimized_xyz = molecule.write("xyz")
  return optimized_xyz

In [37]:
# test

smiles = "CC1=CC(=O)/C(=C(/C)\[O-])/C(=O)O1"

print(openbabel_optimize(smiles, -1, "ghemical"))

19

C         -3.39013       -0.20083       -0.06628
C         -1.90453        0.02615       -0.05303
C         -1.36517        1.24233       -0.17709
C          0.08537        1.42869       -0.16014
O          0.56918        2.54296       -0.27408
C          0.91283        0.23019       -0.00217
C          2.24875        0.32030        0.02270
C          3.19211       -0.84094        0.17948
O          2.84994        1.50170       -0.09577
C          0.17663       -1.04178        0.12305
O          0.77823       -2.09420        0.25849
O         -1.16190       -1.07990        0.09220
H         -3.61129       -1.27233        0.04973
H         -3.85659        0.35430        0.76109
H         -3.80977        0.15179       -1.02015
H         -2.01110        2.11177       -0.29288
H          4.23079       -0.47822        0.16553
H          3.00991       -1.34951        1.13781
H          3.05683       -1.55257       -0.64848



## mopac

In [39]:
# utils
from io import StringIO
from IPython.display import clear_output

# ase / xtb
import ase
from ase.io import read
from ase.calculators.mopac import MOPAC
from ase.optimize import BFGS

def mopac_optimize(xyz_string:str, charge:int, method:str, clear_log=True)->str:
  """
  Description
  -----------
  xtb geometry optimize 함수

  Parameters
  ----------
  - xyz_string (str) : xyz format string
  - charge (int) : molecular total chage
  - clear_log (bool) : clear optimization logging
  - method (str) : supported semi-empirical method
    - supported methods = ['AM1', 'MNDO', 'MNDOD', 'PM3', 'PM6', 'PM6-D3',
                           'PM6-DH+', 'PM6-DH2', 'PM6-DH2X', 'PM6-D3H4',
                           'PM6-D3H4X', 'PMEP', 'PM7', 'PM7-TS', 'RM1']

  Returns
  -------
  - optimized xyz (str)
  """

  # convert xyz format --> ase.Atoms
  mol = ase.io.read(StringIO(xyz_string), format="xyz")

  # set XTB calculator
  mol.calc = MOPAC(method=method, charge=charge )

  # geometry optimize
  optimizer = BFGS(mol)
  optimizer.run()

  # get xyz format string
  with StringIO() as output:
    ase.io.write(output, mol, format="xyz")
    opt_xyz = output.getvalue()

  # clear cell output
  if clear_log:
    clear_output()

  return opt_xyz

In [40]:
# test

xyz_input = """4

B     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(mopac_optimize(xyz_input, charge=0, method="PM6-D3"))

4

B       0.183532316822301      1.843732602886569      1.049877548336631
H      -0.154961793768968      0.913605099078971      1.692628681773127
H      -0.187596385099465      2.926506751249641      1.336413956175056
H       0.893028859744355      1.690767545074082      0.119598816107866



## torchani

In [41]:
# utils
from io import StringIO
from IPython.display import clear_output

from ase.optimize import BFGS
import torchani
import ase
from ase import Atoms
from ase.io import read

def torchani_optimize(xyz_string:str, charge:int, model:str, clear_log=True)->str:
  """
  Description
  -----------
  torchani geometry optimize 함수

  Parameters
  ----------
  - xyz_string (str) : xyz format string
  - charge (int) : molecular total chage
  - model (str) : torchani ML potential model
  - clear_log (bool) : clear optimization logging

  Supported models
  ----------------
  - ani1ccx :  CCSD(T)*/CBS (DPLNO-CCSD(T)) | HCNO
  - ani1x   :  wB97X/6-31G(d)               | HCNO
  - ani2x   :  wB97X/6-31G(d)               | HCNOFSCl

  Returns
  -------
  - optimized xyz (str)
  """
  # convert xyz format --> ase.Atoms
  mol = ase.io.read(StringIO(xyz_string), format="xyz")

  # check compatibility between the torchani model and the elements in the molecule
  model = model.lower()
  supporting_elements = {
      "ani1ccx" : {'H', 'C', 'N', 'O'},
      "ani1x"   : {'H', 'C', 'N', 'O'},
      "ani2x"   : {'H', 'C', 'N', 'O', 'F', 'S', 'Cl'}}

  elements_in_mol = set(mol.get_chemical_symbols())
  unsupported_elements = elements_in_mol - supporting_elements[model]
  if len(unsupported_elements) != 0:
    print(f"{unsupported_elements} are not compatible with the {model} model")
    return None

  # torchani supports only neutral compounds
  if charge != 0:
    print("torchani supports only neutral molecule")
    return None

  # set torchani calculator
  if model == "ani1ccx":
    mol.calc = torchani.models.ANI1ccx().ase()
  elif model == "ani1x":
    mol.calc = torchani.models.ANI1x().ase()
  elif model == "ani2x":
    mol.calc = torchani.models.ANI2x().ase()
  else:
    raise ValueError("Not available model")

  # geometry optimize
  optimizer = BFGS(mol)
  optimizer.run()

  # get xyz format string
  with StringIO() as output:
    ase.io.write(output, mol, format="xyz")
    opt_xyz = output.getvalue()

  # clear cell output
  if clear_log:
    clear_output()

  return opt_xyz

In [42]:
# test

xyz_input = """4

B     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(torchani_optimize(xyz_input, charge=0, model="ani1ccx"))

{'B'} are not compatible with the ani1ccx model
None


In [43]:
# test

xyz_input = """4

N     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(torchani_optimize(xyz_input, charge=0, model="ani1ccx"))

4

N      -0.039554717139012      1.812747246639377      0.876698051864045
H      -0.026082400848078      1.112520441426448      1.603849091973325
H      -0.037526407039161      2.712379761560234      1.336168958685682
H       0.837166522664159      1.736964545321463      0.381802897982090



## xtb

In [24]:
# utils
from io import StringIO
from IPython.display import clear_output

# ase / xtb
import ase
from ase.io import read
from xtb_ase import XTB
from ase.optimize import BFGS

def xtb_optimize(xyz_string:str, charge:int, method:str,
                 spinpol:bool|None=None, uhf:int=0, clear_log=True)->str:
  """
  Description
  -----------
  xtb geometry optimize 함수

  Parameters
  ----------
  - xyz_string (str) : xyz format string
  - charge (int) : molecular total chage
  - method (str) : xtb method
    - supported methods : ["gfn0-xtb", "gfn1-xtb", "gfn2-xTB", "gfn-ff"]
  spinpol ( bool | None ) : number of unpaired electrons
  uhf (int) : whether to use spin-polarized xTB ( see :  https://github.com/Andrew-S-Rosen/xtb_ase/blob/main/src/xtb_ase/calculator.py )
  clear_log (bool) : clear optimization logging

  Returns
  -------
  - optimized xyz (str)
  """

  # convert xyz format --> ase.Atoms
  mol = ase.io.read(StringIO(xyz_string), format="xyz")

  # set XTB calculator
  mol.calc = XTB(method=method, charge=charge, spinpol=spinpol, uhf=uhf)

  # geometry optimize
  optimizer = BFGS(mol)
  optimizer.run()

  # get xyz format string
  with StringIO() as output:
    ase.io.write(output, mol, format="xyz")
    opt_xyz = output.getvalue()

  # clear cell output
  if clear_log:
    clear_output()

  return opt_xyz

In [25]:
# test

xyz_input = """4

B     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(xtb_optimize(xyz_input, charge=0, method="gfn2-xTB"))

4

B       0.184806400637684      1.842624356253694      1.049467580258195
H      -0.157990418311179      0.926768883590662      1.682305732999112
H      -0.184077438630232      2.910182964724403      1.331301749208673
H       0.891264456303727      1.695035795431239      0.135443937534018



## aimnet2

In [51]:
from aimnet2ase import aimnet2_optimize as aimnet2_optimize_

def aimnet2_optimize(xyz_string: str, charge: int, model: str, clear_log:bool=True):
  return aimnet2_optimize_(xyz_string, charge, model, clear_log)

In [52]:
# test

xyz_input = """4

N     -0.045012    1.759938    0.842995
H     -0.081846    1.135119    1.675327
H     -0.095049    2.764070    1.441421
H      0.955910    1.715485    0.238776
"""
print(aimnet2_optimize(xyz_input, charge=0, model="wb97m-d3"))

4

N      -0.036423140501540      1.813190432568083      0.879421414940292
H      -0.028234171783795      1.109123957521246      1.604805087875881
H      -0.040044284735379      2.715917564679369      1.336586058814711
H       0.838704590701828      1.736380039819962      0.377706440435397

